In [17]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def is_close(self, other, tolerance):
        return abs(self.x - other.x) < tolerance and abs(self.y - other.y) < tolerance

In [18]:
class Edge:
    def __init__(self, p1, p2, x_speed=0, y_speed=0):
        self.p1 = p1
        self.p2 = p2
        self.x_speed = x_speed
        self.y_speed = y_speed

    def is_point_on_edge(self, point):
        return point.x >= min(self.p1.x, self.p2.x) and point.x <= max(self.p1.x, self.p2.x) and point.y >= min(self.p1.y, self.p2.y) and point.y <= max(self.p1.y, self.p2.y)

In [19]:
class Car:
    def __init__(self, name, position, speed = 1):
        self.name = name
        self.position = position
        self.visited_intersection = False
        self.speed = speed

    def move(self, x, y):
        self.position.x += x
        self.position.y += y

    def is_at_intersection(self):
        if (self.position.x == 0 and self.position.y > -2 and self.position.y <= 0) \
            or (self.position.y == 0 and self.position.x > -2 and self.position.x < 2):
            self.visited_intersection = True
            return True
        return False

In [20]:
import sys

class Controller:
    def __init__(self, cars, edges, speed_reduction_factor=0.5):
        self.cars = cars
        self.edges = edges
        self.time = 0
        self.intersection_block_times = {}
        self.speed_reduction_factor = speed_reduction_factor
        self.edges_with_reduced_speed = []
        # print("initialized with srf", speed_reduction_factor)

    def check_for_collision(self):
        for car1 in self.cars:
            for car2 in self.cars:
                if car1.name == car2.name:
                    continue
                if car1.position.is_close(car2.position, 0.5):
                    # print(f"Collision detected between {car1.name} and {car2.name}")
                    return True

    def predict_collision_at_intersection(self, car):
        car_edge = self.get_edge(car)
        # print(car_edge.x_speed, car_edge.y_speed)
        time_to_intersection = self.time + (abs(car.position.x) + abs(car.position.y) ) / max(car_edge.x_speed, car_edge.y_speed)
        # print(car.name, "tti", time_to_intersection, self.speed_reduction_factor)
        for car_name in self.intersection_block_times:
            if car_name == car.name:
                continue
            intersection_block_time = self.intersection_block_times[car_name]
            if intersection_block_time[0] <= time_to_intersection <= intersection_block_time[1]:
                # print(f"Car {car.name} travelling at speed {car_edge.x_speed + car_edge.y_speed} is blocked by car {car_name} from time {intersection_block_time[0]} to {intersection_block_time[1]}")
                return True
        if not self.intersection_block_times.get(car.name):
            self.intersection_block_times[car.name] = [time_to_intersection, time_to_intersection + 2]
            # print(f"intersection reserved for {car.name} from {time_to_intersection} to {time_to_intersection+2}")
        return False

    def move_car(self, car):
        car_edge = self.get_edge(car)
        car.move(car_edge.x_speed, car_edge.y_speed)
        if self.predict_collision_at_intersection(car):
            # print(f"Possible collision at intersection for car {car.name}")
            if car_edge in self.edges_with_reduced_speed:
              return self.check_for_collision()
            self.edges_with_reduced_speed.append(car_edge)
            car_edge.x_speed = car_edge.x_speed - self.speed_reduction_factor if car_edge.x_speed > 0 else car_edge.x_speed
            car_edge.y_speed = car_edge.y_speed - self.speed_reduction_factor if car_edge.y_speed > 0 else car_edge.y_speed
            # print("new speeds for", car.name, car_edge.x_speed, car_edge.y_speed, "after reducing by", self.speed_reduction_factor)
            if self.predict_collision_at_intersection(car):
              return True
        return self.check_for_collision()


    def get_edge(self, car):
        for edge in self.edges:
            if edge.is_point_on_edge(car.position):
                return edge
        raise Exception(f"Car {car.name} is not on any edge while at ({car.position.x}, {car.position.y})")

    def print_status(self, car):
      car_edge = self.get_edge(car)
      print("car", car.name, "is at", car.position.x, car.position.y, "with speed", car_edge.x_speed, car_edge.y_speed)

    def simulate(self):
        while True:
            if len(self.cars) == 0:
                # print("All cars have left the intersection")
                break
            for car in self.cars:
                self.print_status(car)
                if self.move_car(car):
                    return -1
                if not car.is_at_intersection() and car.visited_intersection:
                    self.cars.remove(car)
                    # print(f"{car.name} has left the intersection")
                    if len(self.cars) == 0:
                        # print("All cars have left the intersection")
                        return self.time
            self.time += 0.1
            # print("Time: ", self.time)

In [21]:
# edges = [
#     # south to north road with two edges
#     Edge(Point(0, -50), Point(0, -40), 0, 1),
#     Edge(Point(0, -40), Point(0, 0), 0, 1),
#     Edge(Point(0, 0), Point(0, 40), 0, 1),

#     # west to east road with two edges
#     Edge(Point(-50, 0), Point(-40, 0), 1, 0),
#     Edge(Point(-40, 0), Point(0, 0), 1, 0),
#     Edge(Point(0, 0), Point(40, 0), 1, 0),
# ]

edges = [
    # south to north road with two edges
    Edge(Point(0, -50), Point(0, 40), 0, 1),

    # west to east road with two edges
    Edge(Point(-50, 0), Point(40, 0), 1, 0),
]

cars = [
    Car("A", Point(0, -50)),
    Car("B", Point(-50, 0)),
]

In [22]:
import numpy as np

class QLearningController(Controller):
    def __init__(self, cars, edges, alpha=0.1, gamma=0.6, epsilon=0.1):
        super().__init__(cars, edges)
        self.q_table = np.zeros((101, 101))  # Q-table for speed_reduction_factor values from 0 to 1
        self.alpha = alpha  # Learning rate
        self.gamma = gamma  # Discount factor
        self.epsilon = epsilon  # Exploration rate

    def simulate(self):
      self.update_q_table()

    def update_q_table(self):
      for i in range(101):
          reward = self.simulate_with_speed_reduction(i/100)
          for j in range(101):
              # print("reward", reward, "for", i/100)
              next_max = np.max(self.q_table[i])
              self.q_table[i, j] = (1 - self.alpha) * self.q_table[i, j] + self.alpha * (reward + self.gamma * next_max)


    def simulate_with_speed_reduction(self, speed_reduction_factor):
      if speed_reduction_factor == 1:
        return -1000
      controller_copy = self.copy_controller(speed_reduction_factor)
      controller_copy.speed_reduction_factor = speed_reduction_factor
      total_time_taken = controller_copy.simulate()
      if total_time_taken == -1:
          return -1000  # Penalize collision heavily
      return -total_time_taken  # Negative total time taken to minimize delay

    def copy_controller(self, speed_reduction_factor):
        cars_copy = [Car(car.name, Point(car.position.x, car.position.y)) for car in self.cars]
        edges_copy = [Edge(edge.p1, edge.p2, edge.x_speed, edge.y_speed) for edge in self.edges]
        return Controller(cars_copy, edges_copy, speed_reduction_factor)

# Example usage
edges = [
    Edge(Point(0, -50), Point(0, 40), 0, 1),  # South to north road
    Edge(Point(-50, 0), Point(40, 0), 1, 0),  # West to east road
]
cars = [
    Car("A", Point(0, -50)),
    Car("B", Point(-50, 0)),
]
controller = QLearningController(cars, edges)
total_time_taken = controller.simulate()
q_table = controller.q_table

car A is at 0 -50 with speed 0 1
car B is at -50 0 with speed 1 0
car A is at 0 -50 with speed 0 1
car B is at -50 0 with speed 1 0
car A is at 0 -50 with speed 0 1
car B is at -50 0 with speed 1 0
car A is at 0 -50 with speed 0 1
car B is at -50 0 with speed 1 0
car A is at 0 -50 with speed 0 1
car B is at -50 0 with speed 1 0
car A is at 0 -49 with speed 0 1
car B is at -49 0 with speed 0.96 0
car A is at 0 -48 with speed 0 1
car B is at -48.04 0 with speed 0.96 0
car A is at 0 -47 with speed 0 1
car B is at -47.08 0 with speed 0.96 0
car A is at 0 -46 with speed 0 1
car B is at -46.12 0 with speed 0.96 0
car A is at 0 -45 with speed 0 1
car B is at -45.16 0 with speed 0.96 0
car A is at 0 -44 with speed 0 1
car B is at -44.199999999999996 0 with speed 0.96 0
car A is at 0 -43 with speed 0 1
car B is at -43.239999999999995 0 with speed 0.96 0
car A is at 0 -42 with speed 0 1
car B is at -42.279999999999994 0 with speed 0.96 0
car A is at 0 -41 with speed 0 1
car B is at -41.319999999

In [23]:
max_q_values = []
for i in range(101):
  print(i/100, ",", max(q_table[i]))
  max_q_values.append(max(q_table[i]))
most_optimal_srf = np.argmax(max_q_values)/100
print("most optimal edge weight", 1-most_optimal_srf)

0.0 , -100.0
0.01 , -100.0
0.02 , -100.0
0.03 , -100.0
0.04 , -0.5499999999999997
0.05 , -0.5499999999999997
0.06 , -0.5599999999999996
0.07 , -0.5599999999999996
0.08 , -0.5699999999999996
0.09 , -0.5799999999999995
0.1 , -0.5799999999999995
0.11 , -0.5899999999999995
0.12 , -0.5899999999999995
0.13 , -0.5999999999999995
0.14 , -0.6099999999999994
0.15 , -0.6099999999999994
0.16 , -0.6199999999999994
0.17 , -0.6299999999999994
0.18 , -0.6399999999999993
0.19 , -0.6399999999999993
0.2 , -0.6499999999999994
0.21 , -0.6599999999999993
0.22 , -0.6699999999999993
0.23 , -0.6799999999999993
0.24 , -0.6899999999999992
0.25 , -0.6899999999999992
0.26 , -0.6999999999999992
0.27 , -0.7099999999999991
0.28 , -0.7199999999999991
0.29 , -0.7299999999999991
0.3 , -0.739999999999999
0.31 , -0.749999999999999
0.32 , -0.7699999999999989
0.33 , -0.7799999999999989
0.34 , -0.7899999999999988
0.35 , -0.7999999999999988
0.36 , -0.8099999999999987
0.37 , -0.8199999999999987
0.38 , -0.8399999999999986
0.39 

In [24]:
optimal_controller = Controller(cars, edges, most_optimal_srf)
delay = optimal_controller.simulate()
delay

car A is at 0 -50 with speed 0 1
car B is at -50 0 with speed 1 0
car A is at 0 -49 with speed 0 1
car B is at -49 0 with speed 0.96 0
car A is at 0 -48 with speed 0 1
car B is at -48.04 0 with speed 0.96 0
car A is at 0 -47 with speed 0 1
car B is at -47.08 0 with speed 0.96 0
car A is at 0 -46 with speed 0 1
car B is at -46.12 0 with speed 0.96 0
car A is at 0 -45 with speed 0 1
car B is at -45.16 0 with speed 0.96 0
car A is at 0 -44 with speed 0 1
car B is at -44.199999999999996 0 with speed 0.96 0
car A is at 0 -43 with speed 0 1
car B is at -43.239999999999995 0 with speed 0.96 0
car A is at 0 -42 with speed 0 1
car B is at -42.279999999999994 0 with speed 0.96 0
car A is at 0 -41 with speed 0 1
car B is at -41.31999999999999 0 with speed 0.96 0
car A is at 0 -40 with speed 0 1
car B is at -40.35999999999999 0 with speed 0.96 0
car A is at 0 -39 with speed 0 1
car B is at -39.39999999999999 0 with speed 0.96 0
car A is at 0 -38 with speed 0 1
car B is at -38.43999999999999 0 with

5.4999999999999964